In [1]:
import numpy as np
import pandas as pd

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

import warnings

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

mpl.rc('font', family='NanumGothic') # 폰트 설정
mpl.rc('axes', unicode_minus=False) # 유니코드에서 음수 부호 설정

# 차트 스타일 설정
sns.set(font="NanumGothic", rc={"axes.unicode_minus":False}, style='darkgrid')
plt.rc("figure", figsize=(10,8))

warnings.filterwarnings("ignore")

# 1. 콘텐츠 기반 필터링
# 추천 시스템은 크게 콘텐츠 기반 필터링 방식과 협업 필터링 방식으로 나뉜다.

# 그리고 협업 필터링 방식은 다시 최근접 이웃 협업 필터링과 잠재 요인 협업 필터링으로 나뉜다.

# 콘텐츠 기반 필터링은 사용자가 특정 아이템을 선호하는 경우, 그와 비슷한 콘텐츠를 가진 다른 아이템을 추천하는 방식이다.

# 예를 들어, 특정 영화에 높은 평점을 줬다면 그 영화의 장르, 배우,감독, 키워드 등이 유사한 다른 영화를 추천한다.

In [2]:
movies =pd.read_csv('C:/Users/Administrator/Desktop/github/Graduation_Project/ML/tmdb_5000_movies.csv')
movies.head(1)


,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800


In [4]:
# 주요 컬럼으로 데이터 프레임 생성
col_lst = ['id', 'title', 'genres', 'vote_average', 'vote_count', 'popularity', 'keywords', 'overview']
movies_df = movies[col_lst]
# id: 아이디

# title: 영화 제목

# genres: 영화가 속한 여러가지 장르

# vote_average: 평균 평점

# vote_count: 평점 투표 수

# popularity: 영화의 인기 정도

# keywords: 영화를 설명하는 주요 키워드 문구

# overview: 영화에 대한 개요 설명

In [6]:
# 컬럼 길이 늘려서 출력
pd.set_option('max_colwidth', 80)
movies_df[['genres','keywords']][:1]

,genres,keywords
0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"": 2964, ""name"": ""future""}, {""id..."


In [8]:
# 옵션 초기화
pd.reset_option("max_colwidth")

In [9]:
movies_df['genres'][0]

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

In [11]:
from ast import literal_eval

movies_df['genres'].apply(literal_eval)[0]

[{'id': 28, 'name': 'Action'},
 {'id': 12, 'name': 'Adventure'},
 {'id': 14, 'name': 'Fantasy'},
 {'id': 878, 'name': 'Science Fiction'}]

In [13]:
from ast import literal_eval

# 문자열을 객체로 변경: 리스트 내의 사전
movies_df['genres'] = movies_df['genres'].apply(literal_eval)
movies_df['keywords'] = movies_df['keywords'].apply(literal_eval)

# 객체에서 name만 추출: 사전 마다 name을 추출
movies_df['genres'] = movies_df['genres'].apply(lambda x : [ dic['name'] for dic in x] )
movies_df['keywords'] = movies_df['keywords'].apply(lambda x : [ dic['name'] for dic in x] )

movies_df[['genres', 'keywords']][:1]

ValueError: malformed node or string: ['Action', 'Adventure', 'Fantasy', 'Science Fiction']

In [14]:
from sklearn.feature_extraction.text import CountVectorizer

# 리스트 객체를 문자열로 변환: 공백으로 구분
movies_df['genres_literal'] = movies_df['genres'].apply(lambda x : (' ').join(x))

# CountVectorizer
count_vect = CountVectorizer(min_df=0, ngram_range=(1,2))
genre_mat = count_vect.fit_transform(movies_df['genres_literal'])

print(genre_mat.shape)

(4803, 276)


In [15]:
from sklearn.metrics.pairwise import cosine_similarity

genre_sim = cosine_similarity(genre_mat, genre_mat)
genre_sim[0]

array([1.        , 0.59628479, 0.4472136 , ..., 0.        , 0.        ,
       0.        ])

In [16]:
def find_sim_movie(df, sim_matrix, title_name, top_n=10):
    
    # 입력한 영화의 index
    title_movie = df[df['title'] == title_name]
    title_index = title_movie.index.values
    
    # 입력한 영화의 유사도 데이터 프레임 추가
    df["similarity"] = sim_matrix[title_index, :].reshape(-1,1)
    
    # 유사도 내림차순 정렬 후 상위 index 추출
    temp = df.sort_values(by="similarity", ascending=False)
    final_index = temp.index.values[ : top_n]
    
    return df.iloc[final_index]

In [17]:
# The Godfather(대부)와 장르별 유사도가 높은 영화 10개
similar_movies = find_sim_movie(movies_df, genre_sim, 'The Godfather', 10)
similar_movies[['title', 'vote_average', "similarity"]]

,title,vote_average,similarity
3636,Light Sleeper,5.7,1.0
892,Casino,7.8,1.0
3866,City of God,8.1,1.0
1243,Mean Streets,7.2,1.0
1370,21,6.5,1.0
4041,This Is England,7.4,1.0
1847,GoodFellas,8.2,1.0
2582,The Place Beyond the Pines,6.8,1.0
1946,The Bad Lieutenant: Port of Call - New Orleans,6.0,1.0
4217,Kids,6.8,1.0


In [18]:
movies_df[['title','vote_average','vote_count']].sort_values('vote_average', ascending=False)[:10]

,title,vote_average,vote_count
3519,Stiff Upper Lips,10.0,1
4247,Me You and Five Bucks,10.0,2
4045,"Dancer, Texas Pop. 81",10.0,1
4662,Little Big Top,10.0,1
3992,Sardaarji,9.5,2
2386,One Man's Hero,9.3,2
2970,There Goes My Baby,8.5,2
1881,The Shawshank Redemption,8.5,8205
2796,The Prisoner of Zenda,8.4,11
3337,The Godfather,8.4,5893


In [19]:
percentile = 0.6
m = movies_df['vote_count'].quantile(percentile)
C = movies_df['vote_average'].mean()

def weighted_vote_average(record):
    v = record['vote_count']
    R = record['vote_average']
    
    return ( (v/(v+m)) * R ) + ( (m/(m+v)) * C )   

movies_df['weighted_vote'] = movies_df.apply(weighted_vote_average, axis=1)

In [20]:
temp = movies_df[['title','vote_average','vote_count','weighted_vote']]
temp.sort_values('weighted_vote', ascending=False)[:10]

,title,vote_average,vote_count,weighted_vote
1881,The Shawshank Redemption,8.5,8205,8.396052
3337,The Godfather,8.4,5893,8.263591
662,Fight Club,8.3,9413,8.216455
3232,Pulp Fiction,8.3,8428,8.207102
65,The Dark Knight,8.2,12002,8.136930
1818,Schindler's List,8.3,4329,8.126069
3865,Whiplash,8.3,4254,8.123248
809,Forrest Gump,8.2,7927,8.105954
2294,Spirited Away,8.3,3840,8.105867
2731,The Godfather: Part II,8.3,3338,8.079586


In [21]:
def find_sim_movie(df, sim_matrix, title_name, top_n=10):
    
    # 입력한 영화의 index
    title_movie = df[df['title'] == title_name]
    title_index = title_movie.index.values
    
    # 입력한 영화의 유사도 데이터 프레임 추가
    df["similarity"] = sim_matrix[title_index, :].reshape(-1,1)
        
    # 유사도와 가중 평점순으로 높은 상위 index 추출 (자기 자신 제거)
    temp = df.sort_values(by=["similarity", "weighted_vote"], ascending=False)
    temp = temp[temp.index.values != title_index]
    
    final_index = temp.index.values[:top_n]
    
    return df.iloc[final_index]

In [22]:
similar_movies = find_sim_movie(movies_df, genre_sim, 'The Godfather', 10)
similar_movies[['title', 'vote_average', "weighted_vote", "similarity"]]

,title,vote_average,weighted_vote,similarity
1881,The Shawshank Redemption,8.5,8.396052,1.0
2731,The Godfather: Part II,8.3,8.079586,1.0
1847,GoodFellas,8.2,7.976937,1.0
3866,City of God,8.1,7.759693,1.0
1663,Once Upon a Time in America,8.2,7.657811,1.0
3887,Trainspotting,7.8,7.591009,1.0
883,Catch Me If You Can,7.7,7.557097,1.0
892,Casino,7.8,7.423040,1.0
281,American Gangster,7.4,7.141396,1.0
4041,This Is England,7.4,6.739664,1.0
